Some plottings for sleep stages and their distribution in our recordings

In [1]:
import numpy as np
import mne
from scipy import signal
from scipy.interpolate import RectBivariateSpline
from mne.filter import resample, filter_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lspopt import spectrogram_lspopt
from matplotlib.colors import Normalize, ListedColormap

import logging
LOGGING_TYPES = dict(DEBUG=logging.DEBUG, INFO=logging.INFO, WARNING=logging.WARNING,
                     ERROR=logging.ERROR, CRITICAL=logging.CRITICAL)
logger = logging.getLogger('yasa')

%matplotlib qt

Load feature object as a dataframe PXNX

In [193]:

# fname = "P17_N2"
# lr = "L"

# df_feat = pd.read_csv(f"feature/{fname} {lr}.csv", index_col=False)

# location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos-merged"
# hypno_30s = np.loadtxt(
#     f"{location_hypno}/{fname.split('_')[0].lower()}{fname.split('_')[1].lower()}_synced.txt"
# )[:, 0]


Load feature object as a dataframe QS

In [196]:
fname = "QS 38"
lr = "R"

df_feat = pd.read_csv(f"feature/{fname} {lr}.csv", index_col=False)

location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos-merged"
hypno_30s = np.loadtxt(f"{location_hypno}/session_{fname.split(' ')[1]}_synced.txt")[:, 0]


Plot histogram of stages distribution and see what is the distribution of different sleep stages. 

In [197]:
stages, counts = np.unique(hypno_30s, return_counts=True)
if -1 not in stages:
    print(f"no artifcat in {fname} {lr}")
    stages = np.insert(stages, 0, -1)
    counts = np.insert(counts, 0, 0)
if (4 not in stages) and (5 in stages):
    print(f"changing 5 to 4 in {fname} {lr}")
    stages[-1] = 4  # change 5 to 4
fig, ax = plt.subplots(figsize = [4, 5])
ax.bar(stages, counts, color="blue")
ax.set(xticks= np.arange(-1, 4+1, 1), xticklabels=["Artifact","Wake","N1","N2","N3","REM"])
plt.xlabel("Sleep stage")
plt.ylabel("Count")
plt.title(f"Sleep stages {fname} {lr}")
plt.tight_layout()
# plt.savefig("stage_distribution_single_6.png")
# plt.savefig("stage_distribution_single_6.svg")
plt.show()

changing 5 to 4 in QS 38 R


Load all hypnograms and count numbers 

In [84]:
p_hypnos = ["P18_N3 L","P18_N2 R","P17_N2 L","P15_N3 L","P15_N2 L","P13_N3 L","P13_N2 L","P12_N3 L","P11_N3 L","P8_N3 L"]

qs_hypnos = ["QS 11 L","QS 12 R","QS 13 L","QS 15 L","QS 16 L","QS 18 R","QS 19 L","QS 20 R","QS 21 R","QS 22 L","QS 24 R","QS 25 R","QS 26 R","QS 28 R","QS 32 R","QS 34 L","QS 35 R","QS 36 R","QS 37 R","QS 38 R","QS 39 R","QS 40 R","QS 41 L","QS 42 R","QS 45 R","QS 46 R","QS 47 R","QS 48 R","QS 49 R","QS 50 R","QS 51 R","QS 52 R","QS 53 R","QS 54 L","QS 55 R","QS 56 L","QS 57 L","QS 58 L","QS 60 L","QS 61 L","QS 63 L","QS 64 L","QS 66 L","QS 68 L","QS 70 L","QS 71 L","QS 72 L","QS 73 L","QS 79 L","QS 82 L","QS 96 L"]

stages_matrix = np.array([[-1,0,1,2,3,4]])

In [85]:
# for PXNX

for i in range(len(p_hypnos)):
    fname, lr = p_hypnos[i].split(" ")
    location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos-merged"
    hypno_30s = np.loadtxt(
        f"{location_hypno}/{fname.split('_')[0].lower()}{fname.split('_')[1].lower()}_synced.txt"
    )[:, 0]
    stages, counts = np.unique(hypno_30s, return_counts=True)
    if -1 not in stages:
        stages = np.insert(stages, 0, -1)
        counts = np.insert(counts, 0, 0)
    if (4 not in stages) and (5 in stages):
        print(f"changing 5 to 4 in {p_hypnos[i]}")
        stages[-1] = 4  # change 5 to 4
    stages_matrix = np.append(stages_matrix, [counts], axis=0)


In [ ]:
# for QS

for i in range(len(qs_hypnos)):
    _, n, lr = qs_hypnos[i].split(" ")
    location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos-merged"
    hypno_30s = np.loadtxt(
        f"{location_hypno}/session_{n}_synced.txt"
    )[:, 0]
    stages, counts = np.unique(hypno_30s, return_counts=True)
    if -1 not in stages:
        stages = np.insert(stages, 0, -1)
        counts = np.insert(counts, 0, 0)
    if (4 not in stages) and (5 in stages):
        print(f"changing 5 to 4 in {qs_hypnos[i]}")
        stages[-1] = 4 # change 5 to 4
    stages_matrix = np.append(stages_matrix,[counts],axis=0)


In [201]:
stages_matrix.shape

(62, 6)

In [200]:
stages_matrix[:6,:]

array([[ -1,   0,   1,   2,   3,   4],
       [  0,  45,  66, 441, 135, 262],
       [211,  24,  51, 402, 178, 164],
       [457, 221, 102, 246,  60,  51],
       [ 60, 352,   6, 261, 216,  31],
       [ 88,  33,  46, 339, 108, 115]])

Plot sleep distribution over all nights

In [162]:
fig, ax = plt.subplots(figsize = [5, 5])
for i in range(stages_matrix.shape[0]-1):
    ax.scatter(stages_matrix[0,:]+np.random.rand()/3,stages_matrix[i,:], alpha=0.2)
ax.set(xticks= np.arange(-1+0.2, 4+1+0.2, 1), xticklabels=["Artifact","Wake","N1","N2","N3","REM"])
box1 = ax.boxplot(stages_matrix, positions=np.arange(-1+0.2, 4+1+0.2, 1), manage_ticks=False, showfliers=False, widths = 0.5, patch_artist = True, medianprops={"linewidth": 2}, whiskerprops={"linewidth": 2.5}, capprops={"linewidth": 2})
c2 = "purple"
# for item in ['boxes', 'whiskers', 'fliers', 'medians', 'caps']:
plt.setp(box1["boxes"], color="mediumvioletred")
plt.setp(box1["whiskers"], color="mediumvioletred")
plt.setp(box1["medians"], color="gold")
plt.title(f"Distribution of sleep stages across {stages_matrix.shape[0]-1} recordings")
plt.xlabel("Sleep stage")
plt.ylabel("Count")
plt.tight_layout()
# plt.savefig("stage_distribution.svg")
# plt.savefig("stage_distribution.png")
plt.show()

